<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/KK_D1_MongoDB_Local_CRUD_operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

In [1]:
from datetime import datetime
import pytz
print('Tested on ',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

Tested on  2023-05-28 08:44:20.880316+05:30
Python 3.10.11
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 20.04.5 LTS
Release:	20.04
Codename:	focal


Based on https://github.com/giffy/MongoDB_PyMongo_Tutorial

#Installs 

In [2]:
#check which version of MongoDB / pymongo is available
!apt-cache policy mongodb
#!pip show pymongo
!pip3 index versions pymongo

mongodb:
  Installed: (none)
  Candidate: 1:3.6.9+really3.6.8+90~g8e540c0b6d-0ubuntu5.3
  Version table:
     1:3.6.9+really3.6.8+90~g8e540c0b6d-0ubuntu5.3 500
        500 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages
        500 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages
     1:3.6.9+really3.6.8+90~g8e540c0b6d-0ubuntu5 500
        500 http://archive.ubuntu.com/ubuntu focal/universe amd64 Packages
pymongo (4.3.3)
Available versions: 4.3.3, 4.3.2, 4.2.0, 4.1.1, 4.1.0, 4.0.2, 4.0.1, 4.0, 3.13.0, 3.12.3, 3.12.2, 3.12.1, 3.12.0, 3.11.4, 3.11.3, 3.11.2, 3.11.1, 3.11.0, 3.10.1, 3.10.0, 3.9.0, 3.8.0, 3.7.2, 3.7.1, 3.7.0, 3.6.1, 3.6.0, 3.5.1, 3.5.0, 3.4.0, 3.3.1, 3.3.0, 3.2.2, 3.2.1, 3.2, 3.1.1, 3.1, 3.0.3, 3.0.2, 3.0.1, 3.0, 2.9.5, 2.9.4, 2.9.3, 2.9.2, 2.9.1, 2.9, 2.8.1, 2.8, 2.7.2, 2.7.1, 2.7, 2.6.3, 2.6.2, 2.6.1, 2.6, 2.5.2, 2.5.1, 2.5, 2.4.2, 2.4.1, 2.4, 2.3, 2.2.1, 2.2, 2.1.1, 2.1, 2.0.1, 2.0, 1.11, 1.10.1, 1.10, 1.9, 1.8.1, 1.8, 1.7,

In [ ]:
#!curl https://pypi.org/pypi/pymongo/json

In [3]:
!apt install mongodb > /dev/null
 
#Local Server Need Not be started for Remote Operations
!service mongodb start
!mongod --version



 * Starting database mongodb
   ...done.
db version v3.6.8
git version: 8e540c0b6db93ce994cc548f000900bdc740f80a
OpenSSL version: OpenSSL 1.1.1f  31 Mar 2020
allocator: tcmalloc
modules: none
build environment:
    distarch: x86_64
    target_arch: x86_64


In [4]:
!pip install pymongo
from pymongo import MongoClient
import pandas as pd
import dateutil

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 20.3 MB/s eta 0:00:00


In [5]:
mongoURI = "mongodb://localhost:27017/"
client = MongoClient(mongoURI)
client.list_database_names() # ['admin', 'local']

['admin', 'config', 'local']

# Create Database objects

In [6]:
# Database PraxisDB will be created only when we create collections, or documents in it
mongoURI = "mongodb://localhost:27017/"
client = MongoClient(mongoURI)
client.list_database_names() 

['admin', 'config', 'local']

In [7]:
# Delete a database
client.drop_database('PraxisDB')

In [8]:
# Create a new collection. We can create the collection or leave to MongoDB to create it as soon as a document is generated.
db = client.PraxisDB
db.create_collection('addresses')       # Optional collection creation 

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'PraxisDB'), 'addresses')

In [9]:
# Show Collections. Query returns a Cursor [ ] ,  list it to see the content
list (db.list_collections())
# empty list '[]' means that there are not collections in database

[{'name': 'addresses',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': Binary(b'5$\x17\xfa|\x89H\xed\x96\xcaN"nNM\xf5', 4)},
  'idIndex': {'v': 2,
   'key': {'_id': 1},
   'name': '_id_',
   'ns': 'PraxisDB.addresses'}}]

In [10]:
# Set the collection to work with
kollection = db.addresses
kollection.insert_one({'name' : 'jordi'})     # Insert one item to create the collection
list (kollection.find())                  # Show the existing collections

[{'_id': ObjectId('6472c761e66fa01a0ca15ec2'), 'name': 'jordi'}]

In [11]:
# Rename a collections
db.addresses.rename('addressbook')
kollection2 = db.addressbook                   # Set the collection to work with
list (db.list_collections())                  # Show the existing collections

[{'name': 'addressbook',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': Binary(b'5$\x17\xfa|\x89H\xed\x96\xcaN"nNM\xf5', 4)},
  'idIndex': {'v': 2,
   'key': {'_id': 1},
   'name': '_id_',
   'ns': 'PraxisDB.addressbook'}}]

In [12]:
# Delete collection
db.drop_collection('addressbook')

{'ns': 'PraxisDB.addressbook', 'nIndexesWas': 1, 'ok': 1.0}

In [13]:
client.list_database_names() 

['admin', 'config', 'local']

# Operations

## Single Row Insert

In [14]:
cDoc1 = {  'name' : 'Prithwis' ,                                     # String 
          'age' : '59',                                             # Integer
          'gender' : 'Male',                                        # String 
          'likes_code' : True,                                      # Boolean
          'address': {
              'street' : 'Panditiya Road',                          # String ( special character with escape \ )
              'number' : 36,                                        # Integer
              'city' : 'Kolkata',                                   # String 
              'floor' : None,                                       # Null 
              'postalcode' : '700029',                              # String containing a number
              },
          'favouritefruits': ['apple','pineapple','orange']         # Array        
       }

In [15]:
# Define database, collection etc
db = client.PraxisDB
# Delete collection
db.drop_collection('AddressBook')
db.create_collection('AddressBook')       # Optional collection creation 
kollection3 = db.AddressBook                   # Set the collection to work with
insert_result = kollection3.insert_one( cDoc1)
insert_result.acknowledged    # Confirms that insert is successful

True

In [16]:
insert_result.inserted_id     # Shows the document ID 

ObjectId('6472c774e66fa01a0ca15ec3')

In [17]:
db.AddressBook.find()  # returns a cursor

In [18]:
#c = db.AddressBook.find()
c = kollection3.find()
for crnt in c:
    print(crnt)

{'_id': ObjectId('6472c774e66fa01a0ca15ec3'), 'name': 'Prithwis', 'age': '59', 'gender': 'Male', 'likes_code': True, 'address': {'street': 'Panditiya Road', 'number': 36, 'city': 'Kolkata', 'floor': None, 'postalcode': '700029'}, 'favouritefruits': ['apple', 'pineapple', 'orange']}


In [19]:
cDoc2 = {  'name' : 'Charanpreet' ,                                     # String 
          'age' : '57',                                             # Integer
          'gender' : 'Male',                                        # String 
          'likes_code' : False,                                      # Boolean
          'address': {
              'street' : None,                          # String ( special character with escape \ )
              'number' : None,                                        # Integer
              'city' : 'Kolkata',                                   # String 
              'floor' : None,                                       # Null 
              'postalcode' : '700001',                              # String containing a number
              },
          'favouritefruits': ['apple','guava','mango']         # Array        
       }

In [20]:
insert_result = kollection3.insert_one( cDoc2)
insert_result.acknowledged    # Confirms that insert is successful

True

In [21]:
#c = db.AddressBook.find()
c = kollection3.find()
for crnt in c:
    print(crnt)

{'_id': ObjectId('6472c774e66fa01a0ca15ec3'), 'name': 'Prithwis', 'age': '59', 'gender': 'Male', 'likes_code': True, 'address': {'street': 'Panditiya Road', 'number': 36, 'city': 'Kolkata', 'floor': None, 'postalcode': '700029'}, 'favouritefruits': ['apple', 'pineapple', 'orange']}
{'_id': ObjectId('6472c785e66fa01a0ca15ec4'), 'name': 'Charanpreet', 'age': '57', 'gender': 'Male', 'likes_code': False, 'address': {'street': None, 'number': None, 'city': 'Kolkata', 'floor': None, 'postalcode': '700001'}, 'favouritefruits': ['apple', 'guava', 'mango']}


In [22]:
cDoc3 = {  'name' : 'Narendra' ,                                     # String 
          'age' : '67',                                             # Integer
          'gender' : 'Male',                                        # String 
          'likes_code' : False,                                      # Boolean
          'address': {
              'street' : None,                          # String ( special character with escape \ )
              'number' : None,                                        # Integer
              'city' : 'Delhi',                                   # String 
              'floor' : None,                                       # Null 
              'postalcode' : '700001',                              # String containing a number
              },
          'favouritefruits': ['apple','guava','mango']         # Array        
       }

cDoc4 = {  'name' : 'Aditya' ,                                     # String 
          'age' : '47',                                             # Integer
          'gender' : 'Male',                                        # String 
          'likes_code' : False,                                      # Boolean
          'address': {
              'street' : None,                          # String ( special character with escape \ )
              'number' : None,                                        # Integer
              'city' : 'Lucknow',                                   # String 
              'floor' : None,                                       # Null 
              'postalcode' : '700001',                              # String containing a number
              },
          'favouritefruits': ['apple','guava','mango']         # Array        
       }

In [23]:
insert_result = kollection3.insert_many([cDoc3,cDoc4])

In [ ]:
#c = db.AddressBook.find()
c = kollection3.find()
for crnt in c:
    print(crnt)

{'_id': ObjectId('642e63fc5b87a3628880cd0f'), 'name': 'Prithwis', 'age': '59', 'gender': 'Male', 'likes_code': True, 'address': {'street': 'Panditiya Road', 'number': 36, 'city': 'Kolkata', 'floor': None, 'postalcode': '700029'}, 'favouritefruits': ['apple', 'pineapple', 'orange']}
{'_id': ObjectId('642e641c5b87a3628880cd10'), 'name': 'Charanpreet', 'age': '57', 'gender': 'Male', 'likes_code': False, 'address': {'street': None, 'number': None, 'city': 'Kolkata', 'floor': None, 'postalcode': '700001'}, 'favouritefruits': ['apple', 'guava', 'mango']}
{'_id': ObjectId('642e64295b87a3628880cd11'), 'name': 'Narendra', 'age': '67', 'gender': 'Male', 'likes_code': False, 'address': {'street': None, 'number': None, 'city': 'Delhi', 'floor': None, 'postalcode': '700001'}, 'favouritefruits': ['apple', 'guava', 'mango']}
{'_id': ObjectId('642e64295b87a3628880cd12'), 'name': 'Aditya', 'age': '47', 'gender': 'Male', 'likes_code': False, 'address': {'street': None, 'number': None, 'city': 'Lucknow',

## Read

In [24]:
list(kollection3.find())

[{'_id': ObjectId('6472c774e66fa01a0ca15ec3'),
  'name': 'Prithwis',
  'age': '59',
  'gender': 'Male',
  'likes_code': True,
  'address': {'street': 'Panditiya Road',
   'number': 36,
   'city': 'Kolkata',
   'floor': None,
   'postalcode': '700029'},
  'favouritefruits': ['apple', 'pineapple', 'orange']},
 {'_id': ObjectId('6472c785e66fa01a0ca15ec4'),
  'name': 'Charanpreet',
  'age': '57',
  'gender': 'Male',
  'likes_code': False,
  'address': {'street': None,
   'number': None,
   'city': 'Kolkata',
   'floor': None,
   'postalcode': '700001'},
  'favouritefruits': ['apple', 'guava', 'mango']},
 {'_id': ObjectId('6472c791e66fa01a0ca15ec5'),
  'name': 'Narendra',
  'age': '67',
  'gender': 'Male',
  'likes_code': False,
  'address': {'street': None,
   'number': None,
   'city': 'Delhi',
   'floor': None,
   'postalcode': '700001'},
  'favouritefruits': ['apple', 'guava', 'mango']},
 {'_id': ObjectId('6472c791e66fa01a0ca15ec6'),
  'name': 'Aditya',
  'age': '47',
  'gender': 'Male'

In [25]:
#list ( collection.find( {'name' : 'Charanpreet' } ))   
list ( kollection3.find( {'name' : 'Prithwis' } ))   

[{'_id': ObjectId('6472c774e66fa01a0ca15ec3'),
  'name': 'Prithwis',
  'age': '59',
  'gender': 'Male',
  'likes_code': True,
  'address': {'street': 'Panditiya Road',
   'number': 36,
   'city': 'Kolkata',
   'floor': None,
   'postalcode': '700029'},
  'favouritefruits': ['apple', 'pineapple', 'orange']}]

In [26]:
list ( kollection3.find( {'address.postalcode' : '700029' } ))             # find, can use one key or more 

[{'_id': ObjectId('6472c774e66fa01a0ca15ec3'),
  'name': 'Prithwis',
  'age': '59',
  'gender': 'Male',
  'likes_code': True,
  'address': {'street': 'Panditiya Road',
   'number': 36,
   'city': 'Kolkata',
   'floor': None,
   'postalcode': '700029'},
  'favouritefruits': ['apple', 'pineapple', 'orange']}]

## Update

In [27]:
## Update an existing document
update_result = kollection3.update_one( 
    {'name' : 'Prithwis'}, 
    {'$set' : { 'age' : 101 }} ) 

list (kollection3.find( {'name' : 'Prithwis' } ))

# Alternative : collection.find_one_and_update( {'name' : Name}, {'$set' : { 'age' : 30 }} ) 

[{'_id': ObjectId('6472c774e66fa01a0ca15ec3'),
  'name': 'Prithwis',
  'age': 101,
  'gender': 'Male',
  'likes_code': True,
  'address': {'street': 'Panditiya Road',
   'number': 36,
   'city': 'Kolkata',
   'floor': None,
   'postalcode': '700029'},
  'favouritefruits': ['apple', 'pineapple', 'orange']}]

In [28]:
## Update an existing document
update_result = kollection3.update_one( 
    {'name' : 'Prithwis'}, 
    {'$set' : { 'name' : 'Prithwis Mukerjee' }} ) 

list (kollection3.find( {'name' : 'Prithwis Mukerjee' } ))

[{'_id': ObjectId('6472c774e66fa01a0ca15ec3'),
  'name': 'Prithwis Mukerjee',
  'age': 101,
  'gender': 'Male',
  'likes_code': True,
  'address': {'street': 'Panditiya Road',
   'number': 36,
   'city': 'Kolkata',
   'floor': None,
   'postalcode': '700029'},
  'favouritefruits': ['apple', 'pineapple', 'orange']}]

## Additional stuff 
https://www.analyticsvidhya.com/blog/2020/02/mongodb-in-python-tutorial-for-beginners-using-pymongo/

#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)